In [ ]:
##code to convert raster data (.bil and .hdr file needed) to netcdf format
##specifically did for the PRISM rainfall data (downloaded from https://prism.oregonstate.edu/recent)
##https://gis.stackexchange.com/questions/379005/using-raster-transform-function-of-rasterio-in-python

#Amanda Sinning

In [2]:
import rasterio
import xarray as xr
import numpy as np
from netCDF4 import Dataset

In [4]:
##if multiple days, loop through days (days for Hurricane Ivan 2004)
days = [14, 15, 16, 17, 18]

for day in days:
    print(day)

14
15
16
17
18


In [6]:
##loop thru the days, and generate/load in the .bil and .hdr file for each day
for day in days: #processes each day thru the nested loop below, one by one
    bil_file = f"/scratch/amanda/prism_data/9_{day}_04/PRISM_ppt_stable_4kmD2_200409{day}_bil.bil" #bil file
    hdr_file = f"/scratch/amanda/prism_data/9_{day}_04/PRISM_ppt_stable_4kmD2_200409{day}_bil.hdr" #.hdr/metadata file

    #open the .bil raster file
    with rasterio.open(bil_file) as src:
        #read data and metadata
        data = src.read(1)  #read the first raster band, which is precip data here, into the 'data' array
        transform = src.transform #map pixel coords to geographic coords (.transform in rasterio accesses the affine matrix)
        ##note: rasterio uses the affine transformation matrix, which details how pixel locations in the raster grid relate to geographic coords 
        crs = src.crs #get coordinate reference system
        nodata = src.nodata 
        width = src.width #width and height of the raster grid
        height = src.height

    #replace nodata values with NaN, so they will be Nan in the netcdf
    if nodata is not None:
        data = np.where(data == nodata, np.nan, data)

    #note the transform matrix: 
    #create coordinate arrays for longitude and latitude based on the transformation from pixel to geographic coords
    ##helpful for figuring out what was needed for lat and lon below: https://gis.stackexchange.com/questions/379005/using-raster-transform-function-of-rasterio-in-python
    #from the post above, if affine matrix is [a,b,c,d,e,f] -> [0,1,2,3,4,5] (use the numerical indicies to access the different elements of 'transform'
    #transform[2] is x coordinate of the top left corner, and transform[0] is the pixel width for each column, so increments by pixel width
    #transform[5] is y-coord of top left corner and transform[5] is the pixel height
    lon = np.arange(transform[2], transform[2] + width * transform[0], transform[0]) #east-west coords of raster grid
    lat = np.arange(transform[5], transform[5] + height * transform[4], transform[4])

    #reverse latitude array if needed (depends on grid orientation)
    #sometimes after transformation, the lat values decrease as go down the array, but we want them in standard north to south order
    #this happens because raster height is usually negative (y dimension is oriented in the opposite direction (upwards) to the raster row direction (downwards))
    if lat[1] < lat[0]:
        lat = lat[::-1]
        data = data[::-1, :]

    #convert data to an xarray dataset - basically setting up the variables and metadata in the .nc file to be saved 
    #define the preci with lat and lon, so it is the correct shape
    
    ds = xr.Dataset(
        {
            "precipitation": (["lat", "lon"], data),
        },
        coords={
            "lon": lon,
            "lat": lat,
        },
        attrs={
            "description": "PRISM precipitation data",
            "units": "mm",
            "crs": crs.to_string() if crs else "Unknown",
        },
    )

    # Save the xarray Dataset to NetCDF format
    output_file = f"PRISM_ppt_stable_4kmD1_200409{day}.nc"
    ds.to_netcdf(output_file)

    print(f"NetCDF file created: {output_file}")


NetCDF file created: PRISM_ppt_stable_4kmD1_20040914.nc
NetCDF file created: PRISM_ppt_stable_4kmD1_20040915.nc
NetCDF file created: PRISM_ppt_stable_4kmD1_20040916.nc
NetCDF file created: PRISM_ppt_stable_4kmD1_20040917.nc
NetCDF file created: PRISM_ppt_stable_4kmD1_20040918.nc


In [10]:
#quickly test and check the correct data was converted
ds = Dataset("PRISM_ppt_stable_4kmD1_20040917.nc")
print(ds)
ds1 = Dataset("PRISM_ppt_stable_4kmD1_20040918.nc")
print(ds1)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: PRISM precipitation data
    units: mm
    crs: OGC:CRS83
    dimensions(sizes): lat(621), lon(1405)
    variables(dimensions): float32 precipitation(lat, lon), float64 lon(lon), float64 lat(lat)
    groups: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    description: PRISM precipitation data
    units: mm
    crs: OGC:CRS83
    dimensions(sizes): lat(621), lon(1405)
    variables(dimensions): float32 precipitation(lat, lon), float64 lon(lon), float64 lat(lat)
    groups: 


In [11]:
ds.variables

{'precipitation': <class 'netCDF4._netCDF4.Variable'>
 float32 precipitation(lat, lon)
     _FillValue: nan
 unlimited dimensions: 
 current shape = (621, 1405)
 filling on,
 'lon': <class 'netCDF4._netCDF4.Variable'>
 float64 lon(lon)
     _FillValue: nan
 unlimited dimensions: 
 current shape = (1405,)
 filling on,
 'lat': <class 'netCDF4._netCDF4.Variable'>
 float64 lat(lat)
     _FillValue: nan
 unlimited dimensions: 
 current shape = (621,)
 filling on}

In [12]:
rain = ds.variables['precipitation']
print(np.nanmax(rain))
print(np.shape(rain))

rain1 = ds1.variables['precipitation']
print(np.nanmax(rain1))
print(np.shape(rain1))

263.469
(621, 1405)
210.044
(621, 1405)
